In [15]:
from functions import *
from architectures import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
run = neptune.init(
    project="neptunka/DLDF",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOWY1NDM3Mi05OTNkLTRmNmItYjMzNy1hMjY2NmZhNWI1YjQifQ==",
)  # your credentials

https://app.neptune.ai/neptunka/DLDF/e/DLDF-11
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [3]:
data_lagged_features = pd.read_csv('lagged.csv')

In [4]:
df_train, df_test, target_mean, target_stdev, target, features = preprocess(data_lagged_features)

In [6]:
set_random_seed(7)

In [5]:
batch_size = 128
sequence_length = 30

train_dataset = SequenceDataset(
    df_train,
    target=target,
    features=features,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    df_test,
    target=target,
    features=features,
    sequence_length=sequence_length
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)
train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([128, 30, 582])
Target shape: torch.Size([128])


In [7]:
params = {
    'learning_rate' : 1e-3,
    'weight_decay' : 1e-5,
    'num_hidden_units' : 64,
    'loss_function' : nn.L1Loss(),
    'gamma' : 0,
    'shape' : data_lagged_features.shape,
    'epochs' : 3
}

model = ShallowRegressionLSTM(num_sensors=len(features), hidden_units=params['num_hidden_units'])
loss_function = params['loss_function']
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=params['gamma'])

In [8]:
run['params'] = params
run['model'] = model

In [10]:
epochs = params['epochs']

train(df_train, epochs, train_loader, train_eval_loader,model,
      loss_function, optimizer, scheduler, batch_size, target_mean, target_stdev)

Epoch 0
---------


100%|████████████████████████████████████████████████████████████████████████████████| 652/652 [01:11<00:00,  9.12it/s]


Train loss: 0.34258272791384187


100%|████████████████████████████████████████████████████████████████████████████████| 652/652 [00:32<00:00, 20.29it/s]


Smape = 92.88145053665097

Epoch 1
---------


...

Smape = 91.74707339970558

Epoch 2
---------


100%|████████████████████████████████████████████████████████████████████████████████| 652/652 [01:12<00:00,  8.99it/s]


Train loss: 0.30407631534374563


100%|████████████████████████████████████████████████████████████████████████████████| 652/652 [00:32<00:00, 20.23it/s]

Smape = 92.66364982220121



In [17]:
df_out = get_df_out(model, df_train, df_test, train_eval_loader, test_loader, target, target_mean, target_stdev)
ans = get_submission('test_kaggle.csv', df_out)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.44it/s]


In [18]:
smape(df_out.dropna()['Demand'], df_out.dropna()['Model forecast'])

92.66364982220121

https://www.kaggle.com/c/atsf-fall21-hw3

In [34]:
run['kaggle'] = 86.13685

## -----------------------------------------------------------------------------------------------------------------------------